In [2]:
import sys
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
sys.path.append('./U-2-Net')
from model import U2NET  # U2NET model dosyasını import et

# Ağırlık dosyasının ve giriş resimlerinin yolu
model_dir = 'U-2-Net/saved_models/u2net/u2net.pth'
input_image_path = '../Test Images/test_1.png'  # İşlemek istediğin resmin yolu
output_image_path = '../Test Images/output_image.png'
final_output_path = '../Test Images/cleaned_image.png'

# U^2-Net modelini yükle
def load_model():
    print("Model yükleniyor...")
    net = U2NET(3, 1)  # 3 input channel, 1 output channel
    if torch.cuda.is_available():
        net.load_state_dict(torch.load(model_dir))
        net.to(torch.device('cuda'))
    else:
        net.load_state_dict(torch.load(model_dir, map_location='cpu'))
    net.eval()
    return net

# Resmi işle
def process_image(net):
    print("Resim işleniyor...")
    image = Image.open(input_image_path).convert('RGB')  # RGBA'yı RGB'ye çeviriyoruz
    original_size = image.size  # Orijinal resim boyutunu saklıyoruz
    transform = transforms.Compose([transforms.Resize((320, 320)),
                                    transforms.ToTensor()])
    image_tensor = transform(image).unsqueeze(0)  # Batch boyutunu ekle

    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()

    with torch.no_grad():
        output = net(image_tensor)[0]
    
    output_np = output.squeeze().cpu().data.numpy()
    mask = (output_np > 0.5).astype(np.uint8) * 255

    # Maskeyi kaydet ve yeniden boyutlandır
    output_image = Image.fromarray(mask)
    output_image = output_image.resize(original_size)  # Maskeyi orijinal boyuta geri döndür
    output_image.save(output_image_path)
    
    # Maskeyi görüntüye uygula
    apply_mask_to_image(input_image_path, output_image_path, final_output_path)
    print(f"Arka planı temizlenmiş resim {final_output_path} olarak kaydedildi.")

# Maskeyi asıl görüntüye uygula
def apply_mask_to_image(original_image_path, mask_image_path, output_image_path):
    original_image = Image.open(original_image_path).convert('RGB')
    mask_image = Image.open(mask_image_path).convert('L')  # Maskeyi gri tonlamaya çeviriyoruz

    # Asıl görüntü ile maskeyi birleştiriyoruz
    mask_image_np = np.array(mask_image) / 255.0  # 0 ve 1 arasında bir maske
    original_image_np = np.array(original_image)

    # Maskeyi uygula
    output_image_np = (original_image_np * mask_image_np[:, :, None]).astype(np.uint8)

    # Sonuç görüntüsünü kaydet
    output_image = Image.fromarray(output_image_np)
    output_image.save(output_image_path)

if __name__ == '__main__':
    net = load_model()
    process_image(net)


Model yükleniyor...


C:\Users\zahid\AppData\Local\Temp\ipykernel_18868\1808675494.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(model_dir, map_location='cpu

Resim işleniyor...


C:\Users\zahid\ProgrammingForDataEngineering\E_Ticaret_Hackathon\E_Ticaret_Hackathon\Vision Model\./U-2-Net\model\u2net.py:23: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  src = F.upsample(src,size=tar.shape[2:],mode='bilinear')


Arka planı temizlenmiş resim ../Test Images/cleaned_image.png olarak kaydedildi.


In [3]:
import sys
import os
import torch
import cv2
import numpy as np

# RRDBNet_arch.py dosyasının bulunduğu dizini Python path'ine ekleyin
sys.path.append(os.path.abspath("ESRGAN"))

# RRDBNet_arch modülünü içe aktarın
import RRDBNet_arch as arch

# Modelin yolunu ve giriş dosyasını belirleyin
model_path = 'ESRGAN/models/RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
input_image_path = '../Test Images/cleaned_image.png'  # Girdi görüntüsünün yolu
output_image_path = '../Test Images/upscaled_image.png'  # Çıktı görüntüsünün yolu

# Cihazı belirliyoruz (GPU veya CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ESRGAN modelini yükleme
model = arch.RRDBNet(3, 3, 64, 23, gc=32)  # RRDBNet model yapısı
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

# Tek bir görüntü ile çalışma
def upscale_image(input_image_path, output_image_path):
    # Görüntüyü yükle
    img = cv2.imread(input_image_path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255  # Normalizasyon
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0).to(device)  # Görüntüyü tensora çevir ve GPU'ya taşı

    # Modeli kullanarak süper çözünürlük işlemi
    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()

    # Çıktıyı geri çevir ve kaydet
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()  # Görüntüyü 0-255 aralığına döndür
    cv2.imwrite(output_image_path, output)
    print(f"Yüksek çözünürlüklü görüntü {output_image_path} olarak kaydedildi.")

# Görüntüyü büyütme fonksiyonunu çalıştır
upscale_image(input_image_path, output_image_path)


C:\Users\zahid\AppData\Local\Temp\ipykernel_18868\3632003593.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path), strict=True)


Model path ESRGAN/models/RRDB_ESRGAN_x4.pth. 
Testing...
Yüksek çözünürlüklü görüntü ../Test Images/upscaled_image.png olarak kaydedildi.
